In [1]:
import numpy as np
import random
from random import shuffle
import os
import gensim
import re
import pickle
import tensorflow as tf
import operator
import math
import sys
from copy import deepcopy
from collections import Counter
import string
import fileinput
from nltk.tokenize import sent_tokenize, word_tokenize 
import nltk

from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql.functions import split, explode
from pyspark.sql import functions as func
from pyspark.sql.types import *
from pyspark.ml.linalg import Vectors, DenseVector, VectorUDT

from tensorflow.python.ops import rnn, rnn_cell
from tensorflow.python.framework import ops
from tensorflow.models.rnn.translate import seq2seq_model

## Loading Data

In [ ]:
os.environ["PYSPARK_PYTHON"] = 'python2'

In [ ]:
sc = SparkContext('local[4]')
sqlcontext = SQLContext(sc)

In [2]:
data_dir = '/media/ai2-rey/data_disk/data_sets/SQuAD/'

In [3]:
def load_data(data_dir, prefix):
    filenames = [data_dir + prefix + tag for tag in ['.context', '.question', '.answer', '.span']]

    data = []
    column = []
    line_iter = fileinput.input(filenames)
    for line in line_iter:
        if line_iter.isfirstline():
            data.append(column)
            column = []
        column.append(line.rstrip())
    data.append(column)
    line_iter.close()
    data = data[1:]
    return list(zip(data[0], data[1], data[2], data[3]))

In [4]:
train_data = load_data(data_dir, 'train')
train_cont, train_ques, train_ans, train_span = list(zip(*train_data))

val_data = load_data(data_dir, 'val')
val_cont, val_ques, val_ans, val_span = list(zip(*val_data))

test_data = load_data(data_dir, 'dev')
test_cont, test_ques, test_ans, test_span = list(zip(*test_data))

In [5]:
del train_data, val_data, test_data

In [6]:
print(val_cont[120])
print(val_ques[120])
print(val_ans[120])
print(val_span[120].split())

The Martorana church ( decorated around 1143 ) looked originally even more Byzantine although important parts were later demolished . The dome mosaic is similar to that of the Cappella Palatina , with Christ enthroned in the middle and four bowed , elongated angels . The Greek inscriptions , decorative patterns , and evangelists in the squinches are obviously executed by the same Greek masters who worked on the Cappella Palatina . The mosaic depicting Roger II of Sicily , dressed in Byzantine imperial robes and receiving the crown by Christ , was originally in the demolished narthex together with another panel , the Theotokos with Georgios of Antiochia , the founder of the church .
The Martorana church was likely decorated by the same greek masters who decorated which church ?
the Cappella Palatina
['68', '70']


In [7]:
class QA_Dataset:
  
    def __init__(self, context, question, answer, span,
                 max_vocab_size=None, autopad_context=None, 
                 autopad_ques = None, autopad_answer=None, 
                 context_pad_len=None, ques_pad_len = None, answer_pad_len=None):

        assert autopad_context in {'min', 'max', 'avg'}
        assert autopad_answer in {'min', 'max', 'avg'}
        assert autopad_ques in {'min', 'max', 'avg'}
        
        self.context = context
        self.question = question
        self.answer = answer
        self.span = span
#         span_ids = [i.split() for i in span]
#         t_span = []
#         for s in span_ids:
#             s = [int(c) for c in s]
#             t_span.append(s)
#         self.span = t_span
        
        self.max_vocab_size = max_vocab_size
        self.context_pad_len = context_pad_len
        self.ques_pad_len = ques_pad_len
        self.answer_pad_len = answer_pad_len
        self.__autopad_context = autopad_context
        self.__autopad_ques = autopad_ques
        self.__autopad_answer = autopad_answer
        
        self.vocab = set()
        self.word_counter = dict()
        self.word2id_dict = dict()
        self.id2word_dict = dict()
        
        self.context_ids = None
        self.context_sentences_ids = [] 
        self.question_ids = None
#        self.answer_ids = None
#         self.answer_masks = []
        self.answer_one_hot_labels = []

#         self.context_text = None
#         self.question_text = None
#         self.answer_text = None
        
#         self.num_tokens = None
#         self.max_sent_len_context = 0
#         self.max_num_sentences = 0
#         self.max_sent_len_question = 0
        
        self.__parse_data()
        self.__create_word2id_dict()
        self.__numericize_data()

    def __update_word_counter(self, sequence):
        """ Update word_counter with counts for words in a sentence
        
        Args:
            sequence (list<str>) : list of words in a sequence
        
        """
        for word in sequence:
            self.word_counter[word] = self.word_counter.get(word, 0) + 1
            
    def __create_vocab(self):
        """ Create set of most frequent unique words found in the training data """
        
        if self.max_vocab_size == None:
            self.vocab = set(self.word_counter.keys())
        else:
            self.vocab = set(sorted(self.word_counter, key=self.word_counter.get, reverse=True)[:self.max_vocab_size])
        
    def __shuffle_data(self, data):
        random.shuffle(data)
        return list(zip(*data))
    
    def __parse_data(self):
        
        for idx in range(len(self.context)):
            self.__update_word_counter(self.context[idx].split())
            self.__update_word_counter(self.question[idx].split())
            self.__update_word_counter(self.answer[idx].split())
        
        self.__create_vocab()
        
        shuffle = list(zip(self.context, self.question, self.answer, self.span))        
        self.context, self.question, self.answer, self.span = self.__shuffle_data(shuffle)
    
    def __create_word2id_dict(self):
        
        misc_tokens = ['PAD', 'UNK']
            
        for i, token in enumerate(misc_tokens):
            self.word2id_dict[token] = i
            
        for word in self.vocab:
            self.word2id_dict[word] = len(self.word2id_dict)

        self.vocab |= set(misc_tokens)
        self.id2word_dict = dict(zip(self.word2id_dict.values(), self.word2id_dict.keys()))
        self.num_tokens = len(self.word2id_dict)
    
    def __convert_word2id(self, word):
        try:
            word_id = self.word2id_dict[word]
        except:
            word_id = self.word2id_dict['UNK']
        return word_id
    
    def __apply_padding(self, s, pad_len):
        sequence = s[:]
        if len(sequence) < pad_len:
            sequence += [self.word2id_dict['PAD'] for i in range(pad_len - len(sequence))]
        elif len(sequence) > pad_len:
            sequence = sequence[:pad_len]
        else:
            pass
        return sequence
        
    def __get_seq_length_stats(self, sequences):
        max_len = 0
        min_len = 100000
        avg_len = 0
        for sequence in sequences:
            max_len = max(max_len, len(sequence))
            min_len = min(min_len, len(sequence))
            avg_len += len(sequence)
        avg_len = int(float(avg_len) / len(sequences))
        return min_len, max_len, avg_len

    def __get_max_sequence_lengths(self, context_ids, question_ids, answer_ids):
        min_context_len, max_context_len, avg_context_len = self.__get_seq_length_stats(context_ids)
        min_ques_len, max_ques_len, avg_ques_len = self.__get_seq_length_stats(question_ids)
        min_answer_len, max_answer_len, avg_answer_len = self.__get_seq_length_stats(answer_ids)

        if self.context_pad_len == None:
            if self.__autopad_context != None:
                if self.__autopad_context == 'min':
                    self.context_pad_len = min_context_len
                elif self.__autopad_context == 'max':
                    self.context_pad_len = max_context_len
                elif self.__autopad_context == 'avg':
                    self.context_pad_len = avg_context_len
            else:
                self.context_pad_len = avg_context_len
                
        if self.ques_pad_len == None:
            if self.__autopad_ques != None:
                if self.__autopad_ques == 'min':
                    self.ques_pad_len = min_ques_len
                elif self.__autopad_ques == 'max':
                    self.ques_pad_len = max_ques_len
                elif self.__autopad_ques == 'avg':
                    self.ques_pad_len = avg_ques_len
            else:
                self.ques_pad_len = avg_ques_len
                
        if self.answer_pad_len == None:
            if self.__autopad_answer != None:
                if self.__autopad_answer == 'min':
                    self.answer_pad_len = min_answer_len
                elif self.__autopad_answer == 'max':
                    self.answer_pad_len = max_answer_len
                elif self.__autopad_answer == 'avg':
                    self.answer_pad_len = avg_answer_len
            else:
                self.answer_pad_len = avg_answer_len 
                
        #print('Context length stats: min = {}, max = {}, avg = {}'.format(min_context_len, max_context_len, avg_context_len))
        #print('Context pad length set to: {}'.format(self.context_pad_len))
        
        #print('Question length stats: min = {}, max = {}, avg = {}'.format(min_ques_len, max_ques_len, avg_ques_len))
        #print('Question pad length set to: {}'.format(self.ques_pad_len))        
        
        #print('Answer length stats: min = {}, max = {}, avg = {}'.format(min_answer_len, max_answer_len, avg_answer_len))
        #print('Answer pad length set to: {}'.format(self.answer_pad_len))        
    def __convert_to_one_hot(self, span_ids, num_classes):
        label = []
        for idx in span_ids:
            one_hot = [0 for i in range(num_classes)]
            if idx >= num_classes - 1:
                one_hot[-1] = 1
            else:
                one_hot[idx] = 1
            label.append(one_hot)
        return label
    
    def __convert_to_mask(self, i, a):
        mask = (np.in1d(np.array(self.context_ids[i]), a[i]))*1
        if len(a[i])>1:
            count = 0
            while count < len(mask):
                if mask[count:count+len(a[i])].all():
                    count+=len(a[i])
                else:
                    mask[count] = False
                    count +=1
        return mask
        
    def __tokenize_sentences(self, context_ids, question_ids, answer_ids):
        """ Tokenizes sentences.
        :param raw: dict returned from load_babi
        :param word_table: WordTable
        :return:
        """
        context_tokens = []
        question_tokens = []
        #answer_tokens = []
        
        for i in range(len(context_ids)):
            c_tkn_ids = self.__apply_padding(context_ids[i], self.context_pad_len)
            context_tokens.append(c_tkn_ids)
            q_tkn_ids = self.__apply_padding(question_ids[i], self.ques_pad_len)
            question_tokens.append(q_tkn_ids)
            
            span_ids = [int(x) for x in self.span[i].split()]
            label = self.__convert_to_one_hot(span_ids, self.context_pad_len)
            self.answer_one_hot_labels.append(label)
            
            if i%100 == 0:
                print(i)
            #a_tkn_ids = self.__apply_padding(answer_ids[i], self.answer_pad_len)
            #answer_tokens.append(a_tkn_ids)
        return context_tokens, question_tokens#, answer_tokens
        
    def __convert_text2ids(self, context, question, answer):
        """ Tokenizes sentences.
        :param raw: dict returned from load_babi
        :param word_table: WordTable
        :return:
        """
        context_ids = []
        question_ids = []
        answer_ids = []
        for i in range(len(context)):
            c_ids = [self.__convert_word2id(word) for word in context[i].split()]
            context_ids.append(c_ids)
            q_ids = [self.__convert_word2id(word) for word in question[i].split()]
            question_ids.append(q_ids)
            a_ids = [self.__convert_word2id(word) for word in answer[i].split()]
            answer_ids.append(a_ids)            
        return context_ids, question_ids, answer_ids
          
    def __convert_text2words(self, context, question, answer):
        context_words = []
        question_words = []
        answer_words = []
        for i in range(len(context)):
            c_words = context[i].split()
            context_words.append(c_words)
            q_words = question[i].split()
            question_words.append(q_words)
            a_words = answer[i].split()
            answer_words.append(a_words)
        return context_words, question_words, answer_words    

    def __get_max_sentence_length(self, stories):
        num_sentences = 0
        sen_len = 0
        for story in stories: 
            sentences = sent_tokenize(story)
            num_sentences = max(num_sentences, len(sentences))
            word_count = lambda sentence: len(word_tokenize(sentence))
            sen_len = max(sen_len, len((max(sentences, key=word_count)).split()))
        return sen_len, num_sentences
        
    def __context2sentences(self, story):
        story_sentences = []
        curr_sentence = []
        for word_id in story:
            if self.id2word_dict[word_id] == '.':
                if len(curr_sentence) < self.max_sent_len_context:
                    curr_sentence += [0 for i in range(self.max_sent_len_context - len(curr_sentence))]
                elif len(curr_sentence) > self.max_sent_len_context:
                    curr_sentence = curr_sentence[:self.max_sent_len_context]
                story_sentences.append(curr_sentence)
                curr_sentence = []
            else:
                curr_sentence.append(word_id)
            
        return story_sentences
    
    def __numericize_data(self):
        c, q, a = self.__convert_text2ids(self.context, self.question, self.answer)
        
        self.__get_max_sequence_lengths(c, q, a)
        
#         self.context_ids, self.question_ids, self.answer_ids = self.__tokenize_sentences(c, q, a)
        self.context_ids, self.question_ids = self.__tokenize_sentences(c, q, a)
            
#         self.context_text, self.question_text, self.answer_text = self.__convert_text2words(self.context,
#                                                                                             self.question,
#                                                                                             self.answer)
        #self.max_sent_len_context, self.max_num_sentences = self.__get_max_sentence_length(self.context)
        #print("max_sent_len_context: {} max_num_sentences: {}".format(self.max_sent_len_context, self.max_num_sentences))
        #self.max_sent_len_question, ques_num_sentences = self.__get_max_sentence_length(self.question)
        #print("max_sent_len_ques: {} ques_num_sentences: {}".format(self.max_sent_len_question, ques_num_sentences))
        #for c_ids in c:
            #self.context_sentences_ids.append(self.__context2sentences(c_ids))

In [8]:
UNKNOWN = "*UNKNOWN*"

def generate_embedding_files(filename):
    embeddings = {}
    for line in open(filename):
        parts = line.split()
        embeddings[parts[0]] = list(map(float, parts[1:]))
    embedding_size = len(list(embeddings.values())[0])
    embeddings[UNKNOWN] = [0.0 for _ in range(embedding_size)]
    
    words = embeddings.keys()
    embedding_matrix = np.array([embeddings[word] for word in list(embeddings.keys())])
    return words, embedding_matrix

glove_words, glove_embedding_matrix = generate_embedding_files('/media/ai2-rey/data_disk/data_sets/glove.6B/glove.6B.50d.txt')
glove_vocab_lookup = {word: i for i, word  in enumerate(glove_words)}
glove_vocab_size, glove_embedding_size= glove_embedding_matrix.shape

In [9]:
class DataIterator:  
    def __init__(self, data, batch_size):
        self.data = data
        self.batch_size = batch_size
        self.data_iterator = self.make_random_iter()
        
    def next_batch(self):
        try:
            idxs = next(self.data_iterator)
        except StopIteration:
            self.data_iterator = self.make_random_iter()
            idxs = next(self.data_iterator)
            
        batch = [self.data[i] for i in idxs]
        batch_idxs = [idx for idx in idxs]
        return batch, batch_idxs

    def make_random_iter(self):
        splits = np.arange(self.batch_size, len(self.data), self.batch_size)
        it = np.split(np.random.permutation(range(len(self.data))), splits)[:-1]
        return iter(it)

In [10]:
train = QA_Dataset(train_cont, train_ques, train_ans, train_span,
                   context_pad_len = 200, ques_pad_len = 20,
                   autopad_context='max', autopad_ques = 'max', autopad_answer='max')

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [11]:
val = QA_Dataset(val_cont, val_ques, val_ans, val_span,
                 context_pad_len=train.context_pad_len, 
                 ques_pad_len = train.ques_pad_len, 
                 answer_pad_len=train.answer_pad_len,
                 autopad_context='max', autopad_ques='max', autopad_answer='max')

test = QA_Dataset(test_cont, test_ques, test_ans, test_span,
                  context_pad_len=train.context_pad_len, 
                  ques_pad_len = train.ques_pad_len,
                  answer_pad_len=train.answer_pad_len,
                  autopad_context='max', autopad_ques='max', autopad_answer='max')

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200


In [12]:
del train_cont, train_ques, train_ans, train_span
del val_cont, val_ques, val_ans, val_span
del test_cont, test_ques, test_ans, test_span

In [13]:
train_ids = list(zip(train.context_ids, train.question_ids, train.answer_one_hot_labels))
val_ids = list(zip(val.context_ids,val.question_ids,val.answer_one_hot_labels))
test_ids = list(zip(test.context_ids,test.question_ids, test.answer_one_hot_labels))

In [14]:
train_data_iter = DataIterator(train_ids,128)
val_data_iter = DataIterator(val_ids,128)
test_data_iter = DataIterator(test_ids,128)
#deploy_data_iter = DataIterator(train_ids,1)

In [15]:
train.ques_pad_len

20

In [16]:
train.context_pad_len

200

In [30]:
passage_max_len = train.context_pad_len
question_max_len = train.ques_pad_len
#output of lstm is n_steps or passage length by hidden size 
n_hidden = 50 
num_indices = 2
batch_size = 128
n_hidden_layers = 2
learning_rate = 0.001
blend_units = 6
train_iters = 100000
keep_prob = 0.5
display_step = 200
val_interval = 1000
word_dim = 50

In [18]:
glove_embedding_matrix[0]

array([-0.22567 , -0.48367 , -0.28596 ,  0.078052,  0.52416 , -0.092551,
        0.57707 , -0.83893 ,  0.2523  ,  0.67678 , -0.19255 , -0.60286 ,
        1.2789  , -0.27485 ,  0.5941  ,  0.11142 , -0.08224 , -0.46565 ,
       -0.060026, -0.37025 ,  1.244   ,  0.062624, -0.52119 , -0.097129,
       -0.47283 ,  1.409   ,  1.2582  , -0.25652 , -0.041171, -0.98153 ,
       -1.1759  ,  1.2136  , -0.14361 ,  0.95605 ,  0.23754 , -0.24656 ,
        0.28346 ,  0.39402 ,  0.26789 , -0.41279 ,  0.54999 , -0.054683,
        1.0312  ,  0.15117 , -0.38207 ,  0.087894,  0.67466 , -0.22228 ,
        0.19923 ,  0.090407])

In [19]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

In [20]:
passage = tf.placeholder(tf.int32, [batch_size, passage_max_len], name='passage')
question = tf.placeholder(tf.int32,[batch_size, question_max_len], name='question')
#desired_output = tf.placeholder(tf.int32, [num_indices, batch_size, passage_max_len], name='desired_output')
y = tf.placeholder(tf.float32,[num_indices,batch_size,passage_max_len])

dropout_keep_prob = tf.placeholder(tf.float32)
#batch_size = tf.placeholder(tf.int32)

embeddings = tf.constant(glove_embedding_matrix, name='embeddings', dtype=tf.float32)

#preprocessing layer 
passage_embedded = tf.nn.embedding_lookup(embeddings, passage)
question_embedded = tf.nn.embedding_lookup(embeddings, question)

with tf.variable_scope('passage_lstm'):
    passage_cell = tf.nn.rnn_cell.LSTMCell(n_hidden)
    passage_cell = tf.nn.rnn_cell.DropoutWrapper(passage_cell, output_keep_prob=dropout_keep_prob)
    passage_cell = tf.nn.rnn_cell.MultiRNNCell([passage_cell]*n_hidden_layers)
    #H_p shape = batch_size, max_passage_len, n_hidden
    H_p, _ = tf.nn.dynamic_rnn(passage_cell, passage_embedded, dtype=tf.float32) 
    
with tf.variable_scope('question_lstm'):
    question_cell = tf.nn.rnn_cell.LSTMCell(n_hidden)
    question_cell = tf.nn.rnn_cell.DropoutWrapper(question_cell, output_keep_prob=dropout_keep_prob)
    question_cell = tf.nn.rnn_cell.MultiRNNCell([question_cell]*n_hidden_layers)
    H_q, _ = tf.nn.dynamic_rnn(question_cell, question_embedded, dtype=tf.float32)

#Match LSTM layer 
#Weights and bias to compute G
W_q = tf.Variable(tf.truncated_normal(shape=[n_hidden, n_hidden], stddev=0.1))
W_p = tf.Variable(tf.truncated_normal(shape=[n_hidden, n_hidden], stddev=0.1))
W_r = tf.Variable(tf.truncated_normal(shape=[n_hidden, n_hidden], stddev=0.1))
b_p = tf.Variable(tf.truncated_normal(shape=[n_hidden], stddev=0.1))

#Weight and bias to compute a
w= tf.Variable(tf.truncated_normal(shape=[1, n_hidden], stddev=0.1))
#b_alpha shape = 1,none
b_alpha = tf.Variable(tf.constant(0.1, shape = [1]))

#only calculate WH_q once 
#H_q shape = (batch_size&max_passage_len merge, n_hidden)
H_q = tf.reshape(H_q, [-1, H_q.get_shape().as_list()[-1]])
#H_q shape = (n_hidden, batch_size&max_passage_len merge)
H_q = tf.transpose(H_q, [1,0])
WH_q = tf.batch_matmul(W_q, H_q)
WH_q = tf.reshape(WH_q, [n_hidden, question_max_len, -1])
#WH_q shape =(batch_size, n_hidden, max_ques_len)
WH_q = tf.transpose(WH_q, [2,0,1])

H_r_forward = []
H_r_forward_states = []
H_r_backward = []
H_r_backward_states = []

with tf.variable_scope('forward_match_lstm'):
    forward_cell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(n_hidden), output_keep_prob=dropout_keep_prob)
    forward_state = forward_cell.zero_state(batch_size, dtype=tf.float32)
    h = forward_state.h #h shape =(batch_size, n_hidden)
    c = forward_state.c

    for i in range(passage_max_len):
        H_p_i = tf.slice(H_p, [0,i,0], [-1,1,-1])
        H_p_i = tf.transpose(tf.squeeze(H_p_i,1),[1,0])
        WH_p_i = tf.transpose(tf.matmul(W_p, H_p_i), [1,0])
        h = tf.transpose(h,[1,0])
        W_rh = tf.transpose(tf.matmul(W_r, h),[1,0])
        G_tile = tf.tile((WH_p_i + W_rh + b_p), [question_max_len, 1])
        G_tile = tf.reshape(G_tile, [-1, n_hidden, question_max_len])
        G_forward = tf.tanh(WH_q + G_tile)
        G_forward = tf.reshape(G_forward, [n_hidden, -1])
        wG_forward = tf.matmul(w,G_forward)
        a_tile = tf.tile(b_alpha, [question_max_len])
        wG_forward = tf.reshape(tf.squeeze(wG_forward,0),[-1, question_max_len])
        alpha_forward = tf.nn.softmax(wG_forward + a_tile)

        H_q = tf.reshape(H_q,[-1, n_hidden, question_max_len])
        alpha_forward = tf.expand_dims(alpha_forward, 2)
        H_q_alpha_forward = tf.batch_matmul(H_q, alpha_forward)
        H_q_alpha_forward = tf.reshape(H_q_alpha_forward,[-1, n_hidden])
        H_p_i = tf.transpose(H_p_i,[1,0])
        z_forward = tf.concat(1, [H_p_i, H_q_alpha_forward])

        h,forward_state = forward_cell(z_forward, forward_state)
        tf.get_variable_scope().reuse_variables()
        H_r_forward.append(h)
        H_r_forward_states.append(forward_state.c)
        if i%100 == 0:
            print(i)

    H_r_forward = tf.transpose(tf.stack(H_r_forward),[1,0,2])
    
with tf.variable_scope('backward_match_lstm'):
    backward_cell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(n_hidden), output_keep_prob=dropout_keep_prob)
    backward_state = backward_cell.zero_state(batch_size, dtype=tf.float32)
    h = backward_state.h #h shape =(bathc_size, n_hidden)

    for i in reversed(range(passage_max_len)):
        H_p_i = tf.slice(H_p, [0,i,0], [-1,1,-1])
        H_p_i = tf.transpose(tf.squeeze(H_p_i,1),[1,0])
        WH_p_i = tf.transpose(tf.matmul(W_p, H_p_i), [1,0])
        h = tf.transpose(h,[1,0])
        W_rh = tf.transpose(tf.matmul(W_r, h),[1,0])
        G_tile = tf.tile((WH_p_i + W_rh + b_p), [question_max_len, 1])
        G_tile = tf.reshape(G_tile, [-1, n_hidden, question_max_len])
        G_backward = tf.tanh(WH_q + G_tile)
        G_backward = tf.reshape(G_backward, [n_hidden, -1])
        wG_backward = tf.matmul(w,G_backward)
        a_tile = tf.tile(b_alpha, [question_max_len])
        wG_backward = tf.reshape(tf.squeeze(wG_backward,0),[-1, question_max_len])
        alpha_backward = tf.nn.softmax(wG_backward + a_tile)

#         H_q = tf.reshape(H_q, [self.n_hidden, self.n_steps_question, -1])
#         H_q = tf.reshape(H_q, [-1, self.n_steps_question])
#         alpha_backward = tf.transpose(alpha_backward, [1, 0])
#         H_qalpha_backward = tf.matmul(H_q, alpha_backward)
#         H_qalpha_backward = tf.reshape(H_qalpha_backward, [self.n_hidden, -1])
        
        H_q = tf.reshape(H_q,[-1, n_hidden, question_max_len])
        alpha_backward = tf.expand_dims(alpha_backward, 2)
        H_q_alpha_backward = tf.batch_matmul(H_q, alpha_backward)
        H_q_alpha_backward = tf.reshape(H_q_alpha_backward,[-1, n_hidden])
        H_p_i = tf.transpose(H_p_i,[1,0])
        z_backward = tf.concat(1, [H_p_i, H_q_alpha_backward])

        h,backward_state = backward_cell(z_backward, backward_state)
        tf.get_variable_scope().reuse_variables()
        H_r_backward.append(h)
        H_r_backward_states.append(backward_state.c)
        if i%100==0:
            print(i)
    H_r_backward = list(reversed(H_r_backward))
    H_r_backward = tf.transpose(tf.stack(H_r_backward),[1,0,2])
    
    H_r_backward_states = list(reversed(H_r_backward_states))


0
100
100
0


In [ ]:
H_r_forward_try = tf.stack(H_r_forward_states)

In [ ]:
H_r_forward_states[-1]

In [ ]:
H_r_states = tf.concat(2, [H_r_forward_states, H_r_backward_states])
print(H_r_states)

In [ ]:
H_r = tf.concat(2, [H_r_forward, H_r_backward])
print(H_r)

In [ ]:
H_r[-1]

In [21]:
ptr_outputs=  []
ptr_output_dists = [] 

with tf.variable_scope('answer_pointer_lstm'):
    pointer_cell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(n_hidden), output_keep_prob=dropout_keep_prob)
    pointer_state = pointer_cell.zero_state(batch_size, dtype=tf.float32)
    h = pointer_state.h
    c = pointer_state.c
    
    for i in range(num_indices):
        W_1 = tf.Variable(tf.truncated_normal(shape=[n_hidden, blend_units], stddev=0.1))
        W_2 = tf.Variable(tf.truncated_normal(shape=[n_hidden, blend_units], stddev=0.1))
        bias_ptr = tf.Variable(tf.truncated_normal(shape=[batch_size, blend_units], stddev=0.1))
    
        index_predists = []
        enc_portions = []
        v_blend = tf.Variable(tf.truncated_normal(shape=[blend_units,1], stddev=0.1))
        
        for passage_len_idx in range(passage_max_len):
            enc_portion = tf.matmul(H_r_forward_states[passage_len_idx], W_1)
            raw_blend = tf.nn.elu(enc_portion + bias_ptr)
            scaled_blend = tf.matmul(raw_blend, v_blend)
            index_predist = tf.reshape(scaled_blend,(batch_size,))
            
            enc_portions.append(enc_portion)
            index_predists.append(index_predist)
        
        idx_predistribution = tf.transpose(tf.stack(index_predists))
        print(idx_predistribution)
        idx_distribution = tf.nn.softmax(idx_predistribution, dim=-1)
        ptr_output_dists.append(idx_distribution)
        idx = tf.argmax(idx_distribution,1)
        
        emb = tf.nn.embedding_lookup(tf.transpose(passage), idx)
        ptr_output_raw = tf.diag_part(emb)
        
        ptr_output = tf.reshape(ptr_output_raw, (batch_size, 1))
        ptr_outputs.append(ptr_output)

idx_distributions = tf.stack(ptr_output_dists)
print(idx_distributions)
print(idx_distributions[0])

start_label, end_label = tf.split(0,2,y)
start_label_sq = tf.squeeze(start_label, 0)
end_label_sq = tf.squeeze(end_label,0)

#loss = tf.sqrt(tf.reduce_mean(tf.pow(idx_distributions - y, 2.0)))
loss = tf.sqrt(tf.reduce_mean(tf.pow(idx_distributions[0] - start_label_sq, 2.0)))
optimizer = tf.train.AdagradOptimizer(learning_rate).minimize(loss)


print(tf.argmax(idx_distributions[0], axis=1))
print(start_label)
print(start_label_sq)

correct_start_sq = tf.equal(tf.argmax(idx_distributions[0], axis=1), tf.argmax(start_label_sq, axis=1))

start_accuracy_sq = tf.reduce_mean(tf.cast(correct_start_sq, tf.float32))


Tensor("answer_pointer_lstm/transpose:0", shape=(128, 200), dtype=float32)
Tensor("answer_pointer_lstm/transpose_2:0", shape=(128, 200), dtype=float32)
Tensor("stack:0", shape=(2, 128, 200), dtype=float32)
Tensor("strided_slice:0", shape=(128, 200), dtype=float32)
Tensor("ArgMax:0", shape=(128,), dtype=int64)
Tensor("split:0", shape=(1, 128, 200), dtype=float32)
Tensor("Squeeze:0", shape=(128, 200), dtype=float32)


In [ ]:
idx_distributions[0]

In [ ]:
print(tf.argmax(start_label_sq, axis=1))

In [22]:
sess.run(tf.global_variables_initializer())

In [28]:
a,b = val_data_iter.next_batch()
c,d,e = zip(*a)
list(np.array(e).transpose(1,0,2))

[array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ..., 
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]), array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ..., 
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])]

In [32]:
for train_iter in range(train_iters):

    train_iter +=1
    train_batch, _ = train_data_iter.next_batch()
    train_context, train_question, train_answer = zip(*train_batch)
    train_answer = np.array(train_answer).transpose(1,0,2)
    if train_iter == 1:
        train_feed_dict = {passage: train_context,
                           question: train_question,
                           y: train_answer,
                           dropout_keep_prob: 1.0}
        training_loss, start_accuracy = tuple(sess.run([loss, start_accuracy_sq],
                                                       feed_dict=train_feed_dict))

        print("Iter {}, start_accuracy = {:.4f}, train loss = {:.6f}"
              .format(train_iter, start_accuracy, training_loss))

    sess.run(optimizer, feed_dict={passage: train_context,
                                   question: train_question, 
                                   y: train_answer, 
                                   dropout_keep_prob: keep_prob})
    if train_iter % display_step == 0:
        disp_feed_dict = {passage: train_context, 
                          question: train_question, 
                          y: train_answer,
                          dropout_keep_prob: keep_prob
                         }
        training_loss, start_accuracy = tuple(sess.run([loss, start_accuracy_sq],
                                                       feed_dict={passage: train_context, 
                                                                  question: train_question, 
                                                                  y: train_answer,
                                                                  dropout_keep_prob: keep_prob
                                                                 }))
        print("Iter {}, start_accuracy = {:.4f}, train loss = {:.6f}"
              .format(train_iter, start_accuracy, training_loss))
        t = sess.run(idx_distributions[0], feed_dict=disp_feed_dict)
        l = sess.run(start_label_sq, feed_dict=disp_feed_dict)
        print(t[0])
        print(l[0])
        print(np.argmax(t[0]))
        print(np.argmax(l[0]))
    
    if val_data_iter !=None and train_iter%val_interval ==0:
        val_batch, _ = val_data_iter.next_batch()
        val_context, val_question, val_answer = zip(*val_batch)
        val_answer = np.array(val_answer).transpose(1,0,2)
        val_loss, val_start_accuracy = tuple(sess.run([loss, start_accuracy_sq],
                                                      feed_dict={passage: val_context, 
                                                                 question: val_question, 
                                                                 y: val_answer, 
                                                                 dropout_keep_prob: 1.0 
                                                                }))
        print("Iter {}, val_start_accuracy = {:.4f}, val loss = {:.6f}"
              .format(train_iter, val_start_accuracy, val_loss))
        

Iter 1, start_accuracy = 0.0000, train loss = 0.070533
Iter 200, start_accuracy = 0.0000, train loss = 0.070532
[ 0.00490392  0.00489391  0.00488492  0.00488723  0.00485942  0.00487259
  0.0049107   0.00497246  0.00493481  0.0049076   0.00489393  0.00491842
  0.00497048  0.00503315  0.00509935  0.00510728  0.00511322  0.00508387
  0.00510849  0.00513402  0.00512207  0.00506867  0.00502403  0.00503975
  0.00501464  0.00503333  0.00504001  0.0050378   0.00498071  0.00498
  0.00494666  0.00491619  0.0049186   0.00494867  0.00498976  0.00500063
  0.00502159  0.00504874  0.00505286  0.00503058  0.00504848  0.00500009
  0.00502003  0.00504108  0.0050315   0.00506398  0.00500644  0.00504
  0.00498007  0.00504858  0.0050146   0.00505585  0.00502949  0.00505869
  0.00507093  0.0050127   0.00496037  0.00499378  0.00498711  0.00499867
  0.00500694  0.00507314  0.00505453  0.00505048  0.00507539  0.00507492
  0.00509129  0.00511683  0.00510225  0.00508904  0.00503645  0.00499461
  0.0050165   0.00

KeyboardInterrupt: 

In [ ]:
128, 766, 1